In [1]:
# fmt: off
import urllib
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09d_bytes.ipynb"
_nb_title = "Handling image bytes"

_icons=["https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png", "https://www.tensorflow.org/images/colab_logo_32px.png", "https://www.tensorflow.org/images/GitHub-Mark-32px.png", "https://www.tensorflow.org/images/download_logo_32px.png"]
_links=["https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?" + urllib.parse.urlencode({"name": _nb_title, "download_url": "https://github.com/takumiohym/practical-ml-vision-book-ja/raw/master/"+_nb_loc}), "https://colab.research.google.com/github/takumiohym/practical-ml-vision-book-ja/blob/master/{0}".format(_nb_loc), "https://github.com/takumiohym/practical-ml-vision-book-ja/blob/master/{0}".format(_nb_loc), "https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/master/{0}".format(_nb_loc)]
md("""<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="{0}"><img src="{4}"/>Run in Vertex AI Workbench</a></td><td><a target="_blank" href="{1}"><img src="{5}" />Run in Google Colab</a></td><td><a target="_blank" href="{2}"><img src="{6}" />View source on GitHub</a></td><td><a href="{3}"><img src="{7}" />Download notebook</a></td></table><br/><br/>""".format(_links[0], _links[1], _links[2], _links[3], _icons[0], _icons[1], _icons[2], _icons[3]))
# fmt: on

<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?name=Handling+image+bytes&download_url=https%3A%2F%2Fgithub.com%2Ftakumiohym%2Fpractical-ml-vision-book-ja%2Fraw%2Fmaster%2F09_deploying%2F09d_bytes.ipynb"><img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/>Run in Vertex AI Workbench</a></td><td><a target="_blank" href="https://colab.research.google.com/github/takumiohym/practical-ml-vision-book-ja/blob/master/09_deploying/09d_bytes.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a></td><td><a target="_blank" href="https://github.com/takumiohym/practical-ml-vision-book-ja/blob/master/09_deploying/09d_bytes.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td><td><a href="https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/master/09_deploying/09d_bytes.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a></td></table><br/><br/>

# 画像バイトの処理  

このノートブックでは、学習済みでエクスポートされたモデルから始めます。  
このモデルはパブリックの`gs://practical-ml-vision-book-data/flowers_5_trained`に保存されています。

ここでは、ネットワークを介してバイトを直接処理するようにモデルを変更します。そうすることで、クライアントは画像をGoogle Cloud Storageに転送する必要がなくなります。

## 環境設定
以下に使用するプロジェクト名を入力してください。

In [ ]:
PROJECT = 'PROJECT_NAME'

Colab環境を利用する場合、以下で必要なファイルのダウンロード、プロジェクトの設定を行ってください。

In [ ]:
import os

IS_COLAB_BACKEND = 'COLAB_RELEASE_TAG' in os.environ  # this is always set on Colab
if IS_COLAB_BACKEND:
    from google.colab import auth
    auth.authenticate_user()

    # Get files
    BASE_PATH = "https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/main/09_deploying"
    !wget {BASE_PATH}/vertex_deploy.sh
    !sudo chmod 755 ./vertex_deploy.sh

    !gcloud config set project {PROJECT}


## Google Cloud Storageバケットの作成
以下のBUCKETの値を使用可能なGSCバケット名に変更して進めてください。
使用できるバケットがない場合は、以下の二行目のコマンドをコメントアウトし、gsutil mbコマンドを実行して作成してください。

In [ ]:
BUCKET='BUCKET_NAME' # Specify your GCS bucket name
# !gsutil mb -l us-central1 -p {PROJECT} gs://{BUCKET} 

## モデルの読み込み
エクスポートされたモデルを読み込みます。

In [1]:
import os
import shutil
import tensorflow as tf

MODEL_LOCATION='gs://practical-ml-vision-book-data/flowers_5_trained'
model = tf.keras.models.load_model(MODEL_LOCATION)
print(model.summary())

2022-07-17 21:29:10.827657: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-17 21:29:10.827711: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2022-07-17 21:29:13.014347: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-17 21:29:13.014390: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-17 21:29:13.014415: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-direct): /proc/driver/nvidia/version does not exist
2022-07-17 21:29:13.014664: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "flower_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random/center_crop (RandomC  (None, 224, 224, 3)      0         
 rop)                                                            
                                                                 
 random_lr_flip/none (Random  (None, 224, 224, 3)      0         
 Flip)                                                           
                                                                 
 mobilenet_embedding (KerasL  (None, 1280)             2257984   
 ayer)                                                           
                                                                 
 dense_hidden (Dense)        (None, 32)                40992     
                                                                 
 flower_prob (Dense)         (None, 5)                 165       
                                             

In [2]:
IMG_HEIGHT = 345
IMG_WIDTH = 345
IMG_CHANNELS = 3
CLASS_NAMES = 'daisy dandelion roses sunflowers tulips'.split()

def read_from_jpegfile(filename):
    img_bytes = tf.io.read_file(filename)
    return img_bytes
    
def preprocess(img_bytes):
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize_with_pad(img, IMG_HEIGHT, IMG_WIDTH)

In [3]:
filenames = [
    'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg',
    'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg',
    'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg',
    'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg',
    'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg'
]
for filename in filenames:
    img_bytes = read_from_jpegfile(filename)
    img = preprocess(img_bytes)
    img = tf.expand_dims(img, axis=0)
    pred = model.predict(img)
    print(pred)

[[0.20596564 0.61915255 0.0110638  0.08623137 0.07758673]]
[[1.3109286e-05 9.9998438e-01 2.5930876e-07 1.2337833e-06 1.0490534e-06]]
[[9.9508286e-01 3.4755392e-03 7.9892470e-06 1.4052965e-03 2.8355907e-05]]
[[2.8463281e-03 4.0272961e-04 1.6168093e-02 5.3968919e-03 9.7518593e-01]]
[[3.5318535e-06 7.0068648e-07 4.5077931e-02 8.6750697e-07 9.5491701e-01]]


## バイト列を処理するシグネチャのエクスポート
ファイル名でなく、モデルに直接画像ファイルを送れるように、画像のバイト列を処理することのできるシグネチャを追加してエクスポートします。

In [4]:
@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_bytes(img_bytes):
    input_images = tf.map_fn(
        preprocess,
        img_bytes,
        fn_output_signature=tf.float32
    )
    batch_pred = model(input_images) # same as model.predict()
    top_prob = tf.math.reduce_max(batch_pred, axis=[1])
    pred_label_index = tf.math.argmax(batch_pred, axis=1)
    pred_label = tf.gather(tf.convert_to_tensor(CLASS_NAMES), pred_label_index)
    return {
        'probability': top_prob,
        'flower_type_int': pred_label_index,
        'flower_type_str': pred_label
    }

@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_filename(filenames):
    img_bytes = tf.map_fn(
        tf.io.read_file,
        filenames
    )
    result = predict_bytes(img_bytes)
    result['filename'] = filenames
    return result

shutil.rmtree('export', ignore_errors=True)
os.mkdir('export')
model.save('export/flowers_model3',
          signatures={
              'serving_default': predict_filename,
              'from_bytes': predict_bytes
          })

2022-07-17 21:29:28.725077: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: export/flowers_model3/assets


In [5]:
!saved_model_cli show --tag_set serve --dir export/flowers_model3

2022-07-17 21:29:33.095800: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-17 21:29:33.095853: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "from_bytes"
SignatureDef key: "serving_default"


`serving_default`シグネチャには変更はありません。

In [6]:
!saved_model_cli show --tag_set serve --dir export/flowers_model3 --signature_def serving_default

2022-07-17 21:29:36.165361: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-17 21:29:36.165415: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['filename'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall_1:0
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall_1:1
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall_1:2


新たに追加した`from_bytes`シグネチャを確認します

In [7]:
!saved_model_cli show --tag_set serve --dir export/flowers_model3 --signature_def from_bytes

2022-07-17 21:29:39.238504: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-17 21:29:39.238556: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The given SavedModel SignatureDef contains the following input(s):
  inputs['img_bytes'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: from_bytes_img_bytes:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:2
Method na

## imgバイトをネットワーク経由で送信

新しいシグネチャを利用することで、GCSに中間ファイルを置く必要はなくなります。<br>
画像データをローカルにコピーし、直接推論を実行してみましょう。

In [8]:
!gsutil cp gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg /tmp/test.jpg

Copying gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg...
/ [1 files][ 19.4 KiB/ 19.4 KiB]                                                
Operation completed over 1 objects/19.4 KiB.                                     


In [9]:
with open('/tmp/test.jpg', 'rb') as ifp:
    img_bytes = ifp.read()
    serving_fn = tf.keras.models.load_model('./export/flowers_model3').signatures['from_bytes']
    pred = serving_fn(tf.convert_to_tensor([img_bytes]))
    print(pred)

{'flower_type_int': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>, 'flower_type_str': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'daisy'], dtype=object)>, 'probability': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.99508286], dtype=float32)>}


## Vertex AI Prediction へデプロイ

新しいシグネチャを持つモデルをVertex AI Predictionへデプロイします。

デプロイには10分ほどの時間がかかります。

In [10]:
os.environ['BUCKET'] = BUCKET

In [11]:
%%bash
gsutil -m cp -r ./export/flowers_model3 gs://${BUCKET}/flowers_model3

Copying file://./export/flowers_model3/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model3/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model3/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model3/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
/ [4/4 files][ 11.0 MiB/ 11.0 MiB] 100% Done                                    
Operation completed over 4 objects/11.0 MiB.                                     


In [12]:
%%bash
./vertex_deploy.sh \
--endpoint_name=bytes \
--model_name=bytes \
--model_location=gs://${BUCKET}/flowers_model3

Deploying model bytes
Creating bytes endpoint now.
The endpoint_id is 8247172837157109760
Uploading bytes model now.
The model_id is 953439309002702848
Deploying model now


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [2900220921967017984]...
.....done.
Created Vertex AI endpoint: projects/849204435784/locations/us-central1/endpoints/8247172837157109760.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [8200957683382091776]...
.......................................done.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [7960015103317770240]...
............................................................................................................................................................................................................................................................

## 以下のセルを変更する
上記のコマンドから出力された`endpoint_id`を下記にコピーしてください。<br>

In [13]:
# CHANGE THESE TO REFLECT WHERE YOU DEPLOYED THE MODEL
import os
os.environ['ENDPOINT_ID'] = '8247172837157109760' # CHANGE
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = 'us-central1'

In [14]:
%%bash
gsutil cp gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg /tmp/test1.jpg
gsutil cp gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg /tmp/test2.jpg

Copying gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg...
/ [1 files][ 19.4 KiB/ 19.4 KiB]                                                
Operation completed over 1 objects/19.4 KiB.                                     
Copying gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg...
/ [1 files][ 34.6 KiB/ 34.6 KiB]                                                
Operation completed over 1 objects/34.6 KiB.                                     


## HTTPリクエスト
新しい`from_bytes`シグネチャにHTTPリクエストを送ります。

Base64でエンコードされたデータを渡す方法に注意してください。<br>
`img_bytes`キーの下に`b64`キーを定義し、バイト列を値として渡しています。こうすることで、Predictionに使用するTF Servingはこの値を自動的にデコードします。

In [15]:
# Invoke from Python.
import base64
import json
from oauth2client.client import GoogleCredentials
import requests

PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']

def b64encode(filename):
    with open(filename, 'rb') as ifp:
        img_bytes = ifp.read()
        return base64.b64encode(img_bytes).decode()

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:rawPredict".format(
    REGION, PROJECT, REGION, ENDPOINT_ID)
headers = {"Authorization": "Bearer " + token }
data = {
    "signature_name": "from_bytes",
    "instances": [
        {
            "img_bytes": {"b64": b64encode('/tmp/test1.jpg')}
        },
        {
            "img_bytes": {"b64": b64encode('/tmp/test2.jpg')}
        },
    ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{\n    "predictions": [\n        {\n            "flower_type_str": "daisy",\n            "probability": 0.995082855,\n            "flower_type_int": 0\n        },\n        {\n            "flower_type_str": "tulips",\n            "probability": 0.954917,\n            "flower_type_int": 4\n        }\n    ]\n}'


## License
Copyright 2022 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.